In [14]:
import time, random
import requests, re, csv
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup


options = webdriver.ChromeOptions()
options.headless = True
options.add_argument('window-size=2560x1440')
options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36')

headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
    'Accept-Language' : 'ko-KR,ko'
}

driver = webdriver.Chrome(options=options)
# driver = webdriver.Chrome()

# 게시판 : 질문과 답변 (한 페이지에 50개)
# 키워드 : 뉴비
# 최근 한달 뉴비의 어려움 => 7 페이지
# 총 350개의 게시글을 데이터 수집

# URL에 4822번호가 LoastArk의 질문과 답변 게시판을 의미하는 거 같다 (뇌피셜)
# lostark가 wow든 무슨 글자든 4822만 일치하면 강제 매핑
# url = 'https://www.inven.co.kr/board/lostark/4822'
# driver.get(url)

# 검색창 input 태그
# keyword_input = driver.find_element(By.ID, 'sword')
# keyword_input.send_keys("뉴비")
# keyword_input.send_keys(Keys.ENTER)

# 게시글의 번호를 받아올 리스트 선언
# 최근 게시글의 번호부터 들어가므로 내림차순 정렬된다
num=[]

# 7페이지 (대략 한달 정도) 확인
for i in range(1, 8):
    
    # 게시판 페이지 url
    main_url = "https://www.inven.co.kr/board/lostark/4822?name=subject&keyword=%EB%89%B4%EB%B9%84&p={}".format(i)
    
    driver.get(main_url)

    # 게시글 번호 받아오기
    # 게시글 하나 하나에 직접 접근하려고 하니까
    # element is not attached to the page document 에러 발생
    # 이 경우 selenium 객체를 매번 새로 생성하려하니 비효율적인 것 같아
    # 차라리 처음 로딩 때 게시글 번호를 받은 후, 해당 번호로 게시글에 접근하는 것이
    # 더 용이하다고 판단
    board_num = driver.find_elements(By.CLASS_NAME, 'num')  

    for b in board_num:
        num.append(b.text)

    # CLASS_NAME이 num인 태그가 또 존재해서 다른 값도 들어옴
    # 필요없는 값들은 제거
    # 빈 문자열과 최근 게시글 번호에 비해 값이 매우 작은 번호는 다 제외
    num = [x for x in num if x and int(x) > 20]
    
#ex = num[:3]

result = []

for i in num:
    url = 'https://www.inven.co.kr/board/lostark/4822/{}?name=subject&keyword=%EB%89%B4%EB%B9%84'.format(i)

    driver.get(url)
    res = requests.get(url, headers=headers)

    # 에러 발생시 종료
    res.raise_for_status()

    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    time.sleep(random.uniform(1,3))
    
    title = soup.find("div", attrs={"class":"articleTitle"})
    content = soup.find("div", attrs={"id":"powerbbsContent"})  
    
    # 댓글은 엘리먼트가 여러개
    # find_all로 받아오면 html 태그와 non-breaking space 코드(\xa0)가 같이 들어옴
    # 더티 데이터를 최소화 하기 위해 전처리
    comment = soup.find_all("div", attrs={"class":"comment"}) 
    
    comment = str(comment)
    comment = re.sub('<.+?>','',comment,0).strip()
    comment = comment.replace(u'\xa0', u'')
    comment = comment.lstrip('[').rstrip(']')
    
    
    result.append([title.text.strip(), content.text, comment])
    

# csv로 내보내기
today = datetime.today().strftime("%Y_%m_%d")
filename = today + '로아_final.csv'

f = open(filename, 'w', encoding='utf-8-sig', newline='')
writer = csv.writer(f)
title = "title,content,comment".split(',')
    
writer.writerow(title)
writer.writerows(result)

In [16]:
print(len(result))

350
